In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, f1_score
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV
import optuna

In [2]:
# Load data
data_train = np.load('data_train.npy', allow_pickle = True)
data_test = np.load('data_test.npy', allow_pickle = True)
vocab_map = np.load('vocab_map.npy', allow_pickle=True)

# Load labels_train from CSV and extract the 'label' column
labels_train_df = pd.read_csv('label_train.csv')  # Assuming this is your labels file
labels_train = labels_train_df['label'].values  # Extract the labels as a NumPy array

# Convert training data to a DataFrame for visualization
df_train = pd.DataFrame(data_train)

# Add column names using vocab_map
df_train.columns = vocab_map

# Add the target labels to the DataFrame
df_train['TARGETT'] = labels_train

# Separate features and target variable
X = df_train.drop(columns=['TARGETT'])
y = df_train['TARGETT']

# Step 1: Apply TF-IDF Transformation to the Term Count Matrix
print("Applying TF-IDF Transformation...")

tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X)

print("Transformation applied!")

Applying TF-IDF Transformation...
Transformation applied!


In [32]:
# Step 2: Determine Optimal SVD Components Based on Explained Variance
explained_variance_ratio = []
n_components_range = range(4000, 7001, 500)  # Test up to 1000 components in steps of 20 for efficiency

for n in n_components_range:
    svd = TruncatedSVD(n_components=n, random_state=42)
    svd.fit(X_tfidf)
    cumulative_variance = svd.explained_variance_ratio_.sum()
    explained_variance_ratio.append(cumulative_variance)
    print(f"{n} components explain {cumulative_variance:.2%} of the variance")

# Plot Explained Variance to Find Elbow Point
plt.figure()
plt.plot(n_components_range, explained_variance_ratio, marker='o')
plt.xlabel("Number of SVD Components")
plt.ylabel("Cumulative Explained Variance")
plt.title("Optimal Number of Components for SVD")
plt.grid()
plt.show()


4000 components explain 85.01% of the variance
4500 components explain 87.94% of the variance


KeyboardInterrupt: 

In [3]:
# Step 3: Transform Data Using Optimal SVD Components
svd_optimal = TruncatedSVD(n_components=5000, random_state=42)
X_svd_optimal = svd_optimal.fit_transform(X_tfidf)

print("X_svd_otimal found!")

X_svd_otimal found!


In [4]:
# Step 4: Implement different combinations of hyper-parameters for finding a good LightGBM model

# Define a parameter grid for LightGBM
param_grid = {
    'n_estimators': [100, 200, 300],               # Number of boosting rounds
    'learning_rate': [0.01, 0.05, 0.1],            # Step size shrinkage
    'num_leaves': [31, 50, 100],                   # Maximum number of leaves in a tree
    'max_depth': [-1, 10, 20],                     # Maximum depth of the tree (-1 means no limit)
    'min_child_samples': [10, 20, 50],             # Minimum number of samples per leaf
    'min_child_weight': [1e-3, 1e-2, 1e-1],        # Minimum sum of instance weight (hessian) needed in a child
    'subsample': [0.8, 1.0],                       # Fraction of samples used for each tree (boosting round)
    'colsample_bytree': [0.8, 1.0],                # Fraction of features used for each tree
    'lambda_l1': [0, 0.1, 1],                      # L1 regularization
    'lambda_l2': [0, 0.1, 1],                      # L2 regularization
}

# Define the evaluation metric
f1_scorer = make_scorer(f1_score, average='macro')

# Function to train and evaluate a LightGBM model with GPU support
def evaluate_lgbm_model(params, X, y):
    model = LGBMClassifier(**params, n_jobs=-1)  # Use GPU acceleration
    scores = cross_val_score(model, X, y, cv=3, scoring=f1_scorer, n_jobs=-1)
    print(f"Model params: {params}")
    print(f"Mean F1 Score: {scores.mean():.4f}\n")
    return scores.mean()

# Model 1: Default settings with a small number of trees
params_1 = {
    'n_estimators': 100,
    'learning_rate': 0.1,
    'num_leaves': 31,
    'max_depth': -1,
    'random_state': 42,
    'n_jobs': -1
}

# Model 2: Deeper trees with a lower learning rate
params_2 = {
    'n_estimators': 200,
    'learning_rate': 0.05,
    'num_leaves': 50,
    'max_depth': 15,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 42,
    'n_jobs': -1
}

# Model 3: Fewer leaves but higher regularization
params_3 = {
    'n_estimators': 150,
    'learning_rate': 0.05,
    'num_leaves': 20,
    'max_depth': 10,
    'lambda_l1': 0.1,
    'lambda_l2': 0.1,
    'min_child_samples': 20,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'random_state': 42,
    'n_jobs': -1
}

# Evaluate each model
print("Evaluating LightGBM Models...\n")
f1_model_1 = evaluate_lgbm_model(params_1, X_svd_optimal, y)
f1_model_2 = evaluate_lgbm_model(params_2, X_svd_optimal, y)
f1_model_3 = evaluate_lgbm_model(params_3, X_svd_optimal, y)

# Summary of results
results = {
    "Model 1 (Default)": f1_model_1,
    "Model 2 (Deeper Trees)": f1_model_2,
    "Model 3 (Regularization)": f1_model_3
}

print("\nFinal Results Summary:")
for model, score in results.items():
    print(f"{model}: F1 Score = {score:.4f}")

Evaluating LightGBM Models...

Model params: {'n_estimators': 100, 'learning_rate': 0.1, 'num_leaves': 31, 'max_depth': -1, 'random_state': 42, 'n_jobs': -1}
Mean F1 Score: 0.6166

Model params: {'n_estimators': 200, 'learning_rate': 0.05, 'num_leaves': 50, 'max_depth': 15, 'subsample': 0.8, 'colsample_bytree': 0.8, 'random_state': 42, 'n_jobs': -1}
Mean F1 Score: 0.6045

Model params: {'n_estimators': 150, 'learning_rate': 0.05, 'num_leaves': 20, 'max_depth': 10, 'lambda_l1': 0.1, 'lambda_l2': 0.1, 'min_child_samples': 20, 'subsample': 0.9, 'colsample_bytree': 0.9, 'random_state': 42, 'n_jobs': -1}
Mean F1 Score: 0.6144


Final Results Summary:
Model 1 (Default): F1 Score = 0.6166
Model 2 (Deeper Trees): F1 Score = 0.6045
Model 3 (Regularization): F1 Score = 0.6144


In [5]:
# Further dimensionality reduction with GPU acceleration and F1_scores for
# different randomized hyper-parameter combinations

# Define the evaluation metric
f1_scorer = make_scorer(f1_score, average='macro')

# Function to optimize with Optuna
def objective(trial, X, y):
    # Step 1: Optimize `k` for feature selection
    k = trial.suggest_int('k', 500, 5000, step=500)
    
    # Choose the feature selection method
    feature_selection_method = trial.suggest_categorical('method', ['chi2', 'mutual_info'])

    # Automatically switch to mutual_info_classif if the data contains negative values
    if feature_selection_method == 'chi2' and (X < 0).any():
        print("Data contains negative values, switching to mutual_info_classif")
        selector = SelectKBest(mutual_info_classif, k=k)    
        
    elif feature_selection_method == 'chi2':
        selector = SelectKBest(chi2, k=k)
    else:
        selector = SelectKBest(mutual_info_classif, k=k)
    
    X_reduced = selector.fit_transform(X, y)
    
    # Step 2: Optimize LightGBM hyperparameters
    params = {
        'objective': 'binary',
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log = True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),
        'min_child_weight': trial.suggest_float('min_child_weight', 1e-3, 1e-1, log = True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-3, 1.0, log = True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-3, 1.0, log = True),
        'scale_pos_weight': len(y[y == 0]) / len(y[y == 1]),
        'device': 'gpu',  # Use GPU acceleration
        'random_state': 42,
        'n_jobs': -1
    }
    
    model = LGBMClassifier(**params)
    
    # Evaluate using cross-validation
    scores = cross_val_score(model, X_reduced, y, cv=3, scoring=f1_scorer, n_jobs=-1)
    return scores.mean()

# Optimize hyperparameters using Optuna with a progress bar
def optimize_with_optuna(X, y, n_trials=50):
    study = optuna.create_study(direction='maximize')
    
    with tqdm(total=n_trials, desc="Optuna Trials Progress") as pbar:
        def callback(study, trial):
            # Update progress bar on each completed trial
            pbar.update(1)
            # Print the best trial so far
            print(f"Trial {trial.number} completed - F1 Score: {trial.value:.4f}, Best F1 Score: {study.best_value:.4f}")

        study.optimize(lambda trial: objective(trial, X, y), n_trials=n_trials, callbacks=[callback])
    
    print(f"\nBest F1 Score: {study.best_value:.4f}")
    print("Best Parameters:", study.best_params)
    
    # Visualize optimization results
    plot_optimization_history(study).show()
    plot_param_importances(study).show()
    plot_parallel_coordinate(study).show()
    plot_slice(study).show()
    plot_contour(study).show()
    
    return study.best_params

# Load your preprocessed data after applying SVD
print("\nApplying Optuna Optimization on SVD Data...")
best_params = optimize_with_optuna(X_svd_optimal, y, n_trials=50)

# Train final model using the best parameters found
def train_final_model(X, y, params):
    k = params.pop('k')
    method = params.pop('method')
    
    if method == 'chi2':
        selector = SelectKBest(chi2, k=k)
    else:
        selector = SelectKBest(mutual_info_classif, k=k)
    
    X_reduced = selector.fit_transform(X, y)
    model = LGBMClassifier(**params)
    scores = cross_val_score(model, X_reduced, y, cv=3, scoring=f1_scorer, n_jobs=-1)
    print(f"\nFinal Model F1 Score: {scores.mean():.4f}")

print("\nTraining Final Model with Optimized Parameters...")
train_final_model(X_svd_optimal, y, best_params)

[I 2024-11-09 00:00:20,680] A new study created in memory with name: no-name-7db9edad-f876-4f96-afac-57df4be238a7



Applying Optuna Optimization on SVD Data...


Optuna Trials Progress:   2%|█                                                       | 1/50 [08:39<7:04:04, 519.27s/it]

Trial 0 completed - F1 Score: 0.6235, Best F1 Score: 0.6235
Data contains negative values, switching to mutual_info_classif


[I 2024-11-09 00:16:14,275] Trial 1 finished with value: 0.6388731121585408 and parameters: {'k': 2500, 'method': 'chi2', 'n_estimators': 778, 'learning_rate': 0.09849944377720078, 'num_leaves': 81, 'max_depth': 13, 'min_child_samples': 41, 'min_child_weight': 0.012313138723784483, 'subsample': 0.971393047389842, 'colsample_bytree': 0.7559740050109527, 'lambda_l1': 0.04921963431788272, 'lambda_l2': 0.1454697035361137}. Best is trial 1 with value: 0.6388731121585408.
Optuna Trials Progress:   4%|██▏                                                     | 2/50 [15:53<6:15:26, 469.30s/it]

...

Trial 42 completed - F1 Score: 0.6950, Best F1 Score: 0.7002
Data contains negative values, switching to mutual_info_classif


[I 2024-11-09 08:02:56,735] Trial 43 finished with value: 0.6976204984887215 and parameters: {'k': 4000, 'method': 'chi2', 'n_estimators': 784, 'learning_rate': 0.0100669603276488, 'num_leaves': 24, 'max_depth': 15, 'min_child_samples': 16, 'min_child_weight': 0.012461721675270307, 'subsample': 0.5358641577301217, 'colsample_bytree': 0.5048948280662204, 'lambda_l1': 0.9571120770254014, 'lambda_l2': 0.054457001201312215}. Best is trial 14 with value: 0.7001649224770166.
Optuna Trials Progress:  88%|██████████████████████████████████████████████▋      | 44/50 [8:02:36<1:06:17, 662.92s/it]

Trial 43 completed - F1 Score: 0.6976, Best F1 Score: 0.7002
Data contains negative values, switching to mutual_info_classif


[W 2024-11-09 08:06:26,352] Trial 44 failed with parameters: {'k': 4000, 'method': 'chi2'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "D:\miniCondaa\envs\gpu_kaggle1IFT3395\lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\emi_r\AppData\Local\Temp\ipykernel_4648\1199570481.py", line 63, in <lambda>
    study.optimize(lambda trial: objective(trial, X, y), n_trials=n_trials, callbacks=[callback])
  File "C:\Users\emi_r\AppData\Local\Temp\ipykernel_4648\1199570481.py", line 25, in objective
    X_reduced = selector.fit_transform(X, y)
  File "D:\miniCondaa\envs\gpu_kaggle1IFT3395\lib\site-packages\sklearn\utils\_set_output.py", line 316, in wrapped
    data_to_wrap = f(self, X, *args, **kwargs)
  File "D:\miniCondaa\envs\gpu_kaggle1IFT3395\lib\site-packages\sklearn\base.py", line 1101, in fit_transform
    return self.fit(X, y, **fit_params).transform(X)
  File "D:\mi

KeyboardInterrupt: 

In [ ]:
# Further dimensionality reduction with GPU acceleration and F1_scores for
# different randomized hyper-parameter combinations

# Define the evaluation metric
f1_scorer = make_scorer(f1_score, average='macro')

# Function to optimize with Optuna
def objective(trial, X, y):
    # Step 1: Optimize `k` for feature selection
    k = 5000
    
    # Choose the feature selection method
    feature_selection_method = trial.suggest_categorical('method', ['chi2', 'mutual_info'])

    # Automatically switch to mutual_info_classif if the data contains negative values
    if feature_selection_method == 'chi2' and (X < 0).any():
        feature_selection_method = "mutual_info"
        selector = SelectKBest(mutual_info_classif, k=k)    
        
    elif feature_selection_method == 'chi2':
        feature_selection_method = "mutual_info"
        selector = SelectKBest(mutual_info_classif, k=k)
    else:
        selector = SelectKBest(mutual_info_classif, k=k)
    
    X_reduced = selector.fit_transform(X, y)
    
    # Step 2: Optimize LightGBM hyperparameters
    params = {
        'objective': 'binary',
        'n_estimators': trial.suggest_int('n_estimators', 500, 2000),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.011, log = True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 60),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),
        'min_child_weight': trial.suggest_float('min_child_weight', 1e-3, 1e-1, log = True),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.9),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-3, 0.5, log = True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-3, 0.5, log = True),
        'scale_pos_weight': len(y[y == 0]) / len(y[y == 1]),
        'device': 'gpu',  # Use GPU acceleration
        'random_state': 42,
        'n_jobs': -1
    }
    
    model = LGBMClassifier(**params)
    
    # Evaluate using cross-validation
    scores = cross_val_score(model, X_reduced, y, cv=3, scoring=f1_scorer, n_jobs=-1)
    return scores.mean()

# Optimize hyperparameters using Optuna with a progress bar
def optimize_with_optuna(X, y, n_trials=50):
    study = optuna.create_study(direction='maximize')
    
    with tqdm(total=n_trials, desc="Optuna Trials Progress") as pbar:
        def callback(study, trial):
            # Update progress bar on each completed trial
            pbar.update(1)
            # Print the best trial so far
            print(f"Trial {trial.number} completed - F1 Score: {trial.value:.4f}, Best F1 Score: {study.best_value:.4f}")

        study.optimize(lambda trial: objective(trial, X, y), n_trials=n_trials, callbacks=[callback])
    
    print(f"\nBest F1 Score: {study.best_value:.4f}")
    print("Best Parameters:", study.best_params)
    
    # Visualize optimization results
    plot_optimization_history(study).show()
    plot_param_importances(study).show()
    plot_parallel_coordinate(study).show()
    plot_slice(study).show()
    plot_contour(study).show()
    
    return study.best_params

# Load your preprocessed data after applying SVD
print("\nApplying Optuna Optimization on SVD Data...")
best_params = optimize_with_optuna(X_svd_optimal, y, n_trials=50)

# Train final model using the best parameters found
def train_final_model(X, y, params):
    k = params.pop('k')
    method = params.pop('method')
    
    if method == 'chi2':
        selector = SelectKBest(chi2, k=k)
    else:
        selector = SelectKBest(mutual_info_classif, k=k)
    
    X_reduced = selector.fit_transform(X, y)
    model = LGBMClassifier(**params)
    scores = cross_val_score(model, X_reduced, y, cv=3, scoring=f1_scorer, n_jobs=-1)
    print(f"\nFinal Model F1 Score: {scores.mean():.4f}")

print("\nTraining Final Model with Optimized Parameters...")
train_final_model(X_svd_optimal, y, best_params)

[I 2024-11-09 09:31:22,282] A new study created in memory with name: no-name-80b57a1d-2871-4bc8-9fc0-4a1b2fc0ce35



Applying Optuna Optimization on SVD Data...


Optuna Trials Progress:   0%|                                                                   | 0/50 [00:00<?, ?it/s]